# GPT2 based - kaggle + wiki

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install --quiet lightning
! pip install --quiet transformers
! pip install --quiet torch
! pip install --quiet accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 5.2 MB/s eta 0:00:00


In [4]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config, Trainer, TrainingArguments, DataCollatorWithPadding
from torch.utils.data import Dataset, DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
trainset = pd.read_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/train_preprocessed_v2.csv')

train_df, val_df = train_test_split(trainset, test_size=0.3)

In [ ]:
trainset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28738 entries, 0 to 28737
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   text                      28737 non-null  object 
 1   label                     28738 non-null  float64
 2   %unique_word_total        28737 non-null  float64
 3   %stop_word_total          28737 non-null  float64
 4   mean_word_length          28737 non-null  float64
 5   mean_char_count_per_word  28737 non-null  float64
dtypes: float64(5), object(1)
memory usage: 1.3+ MB


In [ ]:
train_df.head()

,text,label,%unique_word_total,%stop_word_total,mean_word_length,mean_char_count_per_word
6359,question automotive landscape changing electri...,1.0,0.465021,0.502058,4.851852,5.851852
7632,think immagination important knowledge think i...,0.0,0.317114,0.553691,4.144295,5.144295
26389,operation diver british codename sabotage oper...,1.0,0.591195,0.377358,5.238994,6.232704
12787,dear state senator much think voting president...,0.0,0.471831,0.478873,4.366197,5.362676
20204,following ships royal navy assigned name calyp...,0.0,0.570370,0.355556,4.459259,5.459259


In [ ]:
train_df['text'] = train_df['text'].fillna('')
val_df['text'] = val_df['text'].fillna('')

## no meta

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, tokenizer, texts, labels):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
train_dataset = TextDataset(tokenizer, train_df['text'].tolist(), train_df['label'].tolist())
val_dataset = TextDataset(tokenizer, val_df['text'].tolist(), val_df['label'].tolist())

In [ ]:
config = GPT2Config.from_pretrained('gpt2', num_labels=2)
config.pad_token_id = tokenizer.eos_token_id

model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch"
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.348700,0.210170
2,0.063500,0.208663
3,0.000100,0.189420


TrainOutput(global_step=7545, training_loss=0.15746776762083553, metrics={'train_runtime': 7505.4952, 'train_samples_per_second': 8.041, 'train_steps_per_second': 1.005, 'total_flos': 1.5768736304726016e+16, 'train_loss': 0.15746776762083553, 'epoch': 3.0})

In [ ]:
model_path = '/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/GPT2 based/GPT2based-kagglewiki-nometa'

trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

In [6]:
# load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = '/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/GPT2 based/GPT2based-kagglewiki-nometa'
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2ForSequenceClassification.from_pretrained(model_path)

model = model.to(device)

In [7]:
torch.save(model, '/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/GPT2 based/GPT2based-kagglewiki-nometa.pth')

In [ ]:
predictions = []
probabilities_list = []
model.eval()

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [ ]:
# validation AUROC
validation_inputs = tokenizer(val_df['text'].tolist(), padding=True, truncation=True, return_tensors="pt", max_length=512)
validation_labels = val_df['label'].values
validation_dataset = TensorDataset(validation_inputs['input_ids'], validation_inputs['attention_mask'], torch.tensor(validation_labels))
validation_dataloader = DataLoader(validation_dataset, batch_size=16, shuffle=False)

for batch in validation_dataloader:
    input_ids, attention_mask, labels = batch

    input_ids = input_ids.to(torch.int64)  # Ensure input IDs are integers
    attention_mask = attention_mask.to(torch.int64)  # Ensure masks are integers
    labels = labels.to(torch.long)  # Ensure labels are long integers

    input_ids = input_ids.to('cuda' if torch.cuda.is_available() else 'cpu')
    attention_mask = attention_mask.to('cuda' if torch.cuda.is_available() else 'cpu')
    labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predictions.extend(probabilities[:, 1].cpu().numpy())  # for class 1

predicted_labels = [int(prob > 0.5) for prob in predictions]  # make class
accuracy = accuracy_score(validation_labels, predicted_labels)
auroc = roc_auc_score(validation_labels, predictions)

print(f"Accuracy of Validation set: {accuracy}")
print(f"AUROC of Validation set: {auroc}")

Accuracy of Validation set: 0.9670610067269775
AUROC of Validation set: 0.9960347112688133


In [ ]:
testset = pd.read_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/new_essay_val_preprocessed.csv')

test_texts = testset['text'].tolist()
true_labels = testset['label'].values

In [ ]:
testset.head()

,text,label,sentence_variation,%stop_word_total,mean_word_length,%adjectives_total
0,sudan conflict human rights year civil war eru...,0,9.078032,0.425243,4.935922,0.082524
1,challenges american foreign service rebuilding...,0,10.024878,0.371251,5.815926,0.099276
2,end era nations habitually realized territoria...,0,23.250937,0.381433,5.890010,0.114026
3,promoting global regional security postcold wa...,0,14.830529,0.418864,5.314402,0.102434
4,mitigation crime honduras microfinance loans s...,0,10.491831,0.423156,5.239754,0.091189


In [ ]:
encoded_inputs = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

In [ ]:
test_dataset = torch.utils.data.TensorDataset(encoded_inputs['input_ids'], encoded_inputs['attention_mask'], torch.tensor(true_labels))
test_dataloader = DataLoader(test_dataset, batch_size=16)

In [ ]:
predictions = []
probabilities_list = []
model.eval()

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [ ]:
for batch in test_dataloader:
    input_ids, attention_mask, labels = batch

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predictions.extend(probabilities[:, 1].cpu().numpy())  # for class 1

In [ ]:
predicted_labels = [int(prob > 0.5) for prob in predictions]  # make class
accuracy = accuracy_score(true_labels, predicted_labels)
auroc = roc_auc_score(true_labels, predictions)

print(f"Accuracy: {accuracy}")
print(f"AUROC: {auroc}")

Accuracy: 0.925
AUROC: 0.9349999999999999


In [ ]:
import matplotlib.pyplot as plt

# calculate the ROC curve points
fpr, tpr, thresholds = roc_curve(true_labels, predictions)

# calculate the AUROC
roc_auc = auc(fpr, tpr)

# plot the ROC curve
plt.figure()
lw = 2  # line width
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()


## with metafeatures

In [ ]:
import pandas as pd
import torch.nn as nn
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from sklearn.preprocessing import StandardScaler
import torch
from transformers import GPT2Model, GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config, Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
trainset = pd.read_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/train_preprocessed_v2.csv')
trainset = trainset.dropna()

In [ ]:
trainset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28737 entries, 0 to 28737
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   text                28737 non-null  object 
 1   label               28737 non-null  float64
 2   sentence_variation  28737 non-null  float64
 3   %stop_word_total    28737 non-null  float64
 4   mean_word_length    28737 non-null  float64
 5   %adjectives_total   28737 non-null  float64
dtypes: float64(5), object(1)
memory usage: 1.5+ MB


In [ ]:
trainset.head()

,text,label,sentence_variation,%stop_word_total,mean_word_length,%adjectives_total
0,school homework clubs become increasingly popu...,1.0,5.291503,0.404908,5.398773,0.055215
1,widely accepted knowledge great source power s...,1.0,6.047842,0.459930,5.236934,0.111498
2,first impressions great power shape interactio...,1.0,9.306841,0.460784,5.250000,0.117647
3,name address city state zip code email address...,1.0,14.740441,0.408935,5.567010,0.096220
4,limiting car usage numerous advantages benefit...,1.0,6.080817,0.404959,5.524793,0.090909


In [ ]:
class CombinedModel(nn.Module):
    def __init__(self, gpt2_model_name, num_meta_features):
        super(CombinedModel, self).__init__()
        # load the pre-trained GPT-2 model
        self.gpt2 = GPT2Model.from_pretrained(gpt2_model_name)

        # define a simple feedforward network for meta-features
        self.meta_feature_ffn = nn.Sequential(
            nn.Linear(num_meta_features, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU()
        )

        # output layer that combines the features from GPT-2 and meta-features
        self.classifier = nn.Linear(self.gpt2.config.n_embd + 16, 2)

    def forward(self, input_ids, attention_mask, meta_features):
        # pass text through GPT-2
        gpt2_output = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        text_embedding = gpt2_output.last_hidden_state[:, 0, :]

        # pass meta-features through the feedforward network
        meta_features_embedding = self.meta_feature_ffn(meta_features)

        # concatenate text and meta-features embeddings
        combined_embedding = torch.cat((text_embedding, meta_features_embedding), dim=1)

        # classify using the combined features
        logits = self.classifier(combined_embedding)

        return logits

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
encodings = tokenizer(trainset['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
labels = trainset['label'].values

meta_features = trainset[['sentence_variation', '%stop_word_total', 'mean_word_length', '%adjectives_total']].values
scaler = StandardScaler()
meta_features = scaler.fit_transform(meta_features)

In [ ]:
# assume encodings is a dictionary with 'input_ids' and 'attention_mask'
input_ids_train, input_ids_val, attention_mask_train, attention_mask_val, train_meta_features, val_meta_features, train_labels, val_labels = train_test_split(
    encodings['input_ids'], encodings['attention_mask'], meta_features, labels, test_size=0.2
)

# after splitting, re-construct the encodings dictionaries
train_encodings = {'input_ids': input_ids_train, 'attention_mask': attention_mask_train}
val_encodings = {'input_ids': input_ids_val, 'attention_mask': attention_mask_val}

In [ ]:
class TextAndMetaDataset(Dataset):
    def __init__(self, encodings, meta_features, labels):
        self.encodings = {key: torch.tensor(val) for key, val in encodings.items()}
        self.meta_features = torch.tensor(meta_features, dtype=torch.float)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __getitem__(self, idx):
        item = {key: self.encodings[key][idx] for key in self.encodings}
        item['meta_features'] = self.meta_features[idx]
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = TextAndMetaDataset(train_encodings, train_meta_features, train_labels)
val_dataset = TextAndMetaDataset(val_encodings, val_meta_features, val_labels)


<ipython-input-12-8cc55d328310>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.encodings = {key: torch.tensor(val) for key, val in encodings.items()}


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False, num_workers=4, pin_memory=True)

model_meta = CombinedModel(gpt2_model_name='gpt2', num_meta_features=4)
model_meta.to(device)

optimizer = Adam(model_meta.parameters(), lr=5e-5)
loss_function = torch.nn.CrossEntropyLoss().to(device)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
from torch.cuda.amp import GradScaler, autocast


scaler = GradScaler()

num_epochs = 3

for epoch in range(num_epochs):
    model_meta.train()

    for batch in train_loader:
        optimizer.zero_grad()

        with autocast():  # Casts operations to mixed precision
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            meta_features = batch['meta_features'].to(device)
            labels = batch['labels'].to(device)

            outputs = model_meta(input_ids, attention_mask, meta_features)
            loss = loss_function(outputs, labels)

        scaler.scale(loss).backward()  # Scales the loss for mixed precision
        scaler.step(optimizer)
        scaler.update()

        # loss.backward()
        # optimizer.step()

    # evaluate on the validation set
    model_meta.eval()
    val_loss = 0
    val_accuracy = 0
    total_val_samples = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            meta_features = batch['meta_features'].to(device)
            labels = batch['labels'].to(device)

            outputs = model_meta(input_ids, attention_mask, meta_features)
            loss = loss_function(outputs, labels)

            val_loss += loss.item()
            predictions = outputs.argmax(dim=1)
            val_accuracy += (predictions == labels).sum().item()
            total_val_samples += labels.size(0)

    val_loss /= len(val_loader)
    val_accuracy /= total_val_samples
    print(f"Epoch {epoch+1}: Validation loss = {val_loss:.4f}, Accuracy = {val_accuracy:.4f}")


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1: Validation loss = 0.5804, Accuracy = 0.6517
Epoch 2: Validation loss = 0.5291, Accuracy = 0.7110
Epoch 3: Validation loss = 0.5055, Accuracy = 0.7347


In [ ]:
torch.save(model_meta, '/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/GPT2 based/GPT2based-kagglewiki-withmeta.pth')


In [ ]:
# load model for score

model_meta = CombinedModel(gpt2_model_name='gpt2', num_meta_features=4)
model_meta = torch.load('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/GPT2 based/GPT2based-kagglewiki-withmeta.pth')
model_meta.to(device)

CombinedModel(
  (gpt2): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (meta_feature_ffn): Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
  )
  (

In [ ]:
testset = pd.read_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/new_essay_val_preprocessed.csv')

In [ ]:
testset.head()

,text,label,sentence_variation,%stop_word_total,mean_word_length,%adjectives_total
0,sudan conflict human rights year civil war eru...,0,9.078032,0.425243,4.935922,0.082524
1,challenges american foreign service rebuilding...,0,10.024878,0.371251,5.815926,0.099276
2,end era nations habitually realized territoria...,0,23.250937,0.381433,5.890010,0.114026
3,promoting global regional security postcold wa...,0,14.830529,0.418864,5.314402,0.102434
4,mitigation crime honduras microfinance loans s...,0,10.491831,0.423156,5.239754,0.091189


In [ ]:
tokenized_test = tokenizer(testset['text'].tolist(), padding=True, truncation=True, return_tensors="pt")
scaler = StandardScaler()
test_meta_features = scaler.fit_transform(testset[['sentence_variation', '%stop_word_total', 'mean_word_length', '%adjectives_total']].values)

In [ ]:
test_meta_features = torch.tensor(test_meta_features, dtype=torch.float)
true_labels = torch.tensor(testset['label'].values, dtype=torch.long)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
test_dataset = TensorDataset(tokenized_test['input_ids'], tokenized_test['attention_mask'], test_meta_features, true_labels)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

# ensure the model is in evaluation mode
model_meta.eval()

# lists to store targets and predictions
actuals = []
predictions = []  # This should be a list where tensor values will be appended
probabilities = []  # This should be a list where tensor values will be appended

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, meta_features, labels = [item.to(device) for item in batch]

        # Forward pass
        outputs = model_meta(input_ids, attention_mask, meta_features)
        outputs = torch.softmax(outputs, dim=1)  # Convert logits to probabilities
        _, predicted = torch.max(outputs, 1)

        # Store predictions and actual labels
        actuals.extend(labels.cpu().numpy())
        predictions.extend(predicted.cpu().numpy())  # Correctly extend with numpy array
        probabilities.extend(outputs[:, 1].cpu().numpy())  # Correctly extend with numpy array

# Calculate Accuracy
accuracy = accuracy_score(actuals, predictions)
print("Accuracy:", accuracy)

# Calculate AUROC
auroc = roc_auc_score(actuals, probabilities)
print("AUROC:", auroc)

Accuracy: 0.9
AUROC: 0.8975
